In [1]:
import plotly.express as px
import torch_geometric as pyg
import torch
import networkx as nx
import pytorch_lightning as pl
import pandas as pd

,id,layout_id,hole_id,set_id,default_placement_role_id
0,1073,1,1134,1,13.0
1,1074,1,1136,1,13.0
2,1075,1,1138,1,13.0
3,1076,1,1140,1,13.0
4,1077,1,1142,1,13.0
...,...,...,...,...,...
3768,4676,8,4172,29,45.0
3769,4677,8,4174,29,45.0
3770,4678,8,4176,29,45.0
3771,4679,8,4178,29,45.0


In [32]:
import sqlite3
conn = sqlite3.connect("data/db.sqlite3")
climbs = pd.read_sql_query("SELECT * FROM climbs", conn)
grades = pd.read_sql_query("SELECT * FROM difficulty_grades", conn)
stats = pd.read_sql_query("SELECT * FROM climb_stats", conn)
holds = pd.read_sql_query("SELECT * FROM holes", conn)
placements = pd.read_sql_query("SELECT * FROM placements", conn)
holds = pd.merge(placements, holds, left_on="hole_id", right_on='id')
holds.set_index("id_x", inplace=True)

In [31]:
holds

,id_x,layout_id,hole_id,set_id,default_placement_role_id,id_y,product_id,name,x,y,mirrored_hole_id,mirror_group
0,1073,1,1134,1,13.0,1134,1,"34,KB2",136,8,0,0
1,1074,1,1136,1,13.0,1136,1,"32,KB2",128,8,0,0
2,1075,1,1138,1,13.0,1138,1,"30,KB2",120,8,0,0
3,1076,1,1140,1,13.0,1140,1,"28,KB2",112,8,0,0
4,1077,1,1142,1,13.0,1142,1,"26,KB2",104,8,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
3768,4676,8,4172,29,45.0,4172,7,"-20,-8",-20,-8,0,0
3769,4677,8,4174,29,45.0,4174,7,"-28,-8",-28,-8,0,0
3770,4678,8,4176,29,45.0,4176,7,"-36,-8",-36,-8,0,0
3771,4679,8,4178,29,45.0,4178,7,"-44,-8",-44,-8,0,0


In [26]:
pd.merge(placements, holds, left_on="hole_id", right_on='hole_id')

,id_x,layout_id,hole_id,set_id,default_placement_role_id,id_y,product_id,name,x,y,mirrored_hole_id,mirror_group
0,1073,1,1134,1,13.0,1134,1,"34,KB2",136,8,0,0
1,1074,1,1136,1,13.0,1136,1,"32,KB2",128,8,0,0
2,1075,1,1138,1,13.0,1138,1,"30,KB2",120,8,0,0
3,1076,1,1140,1,13.0,1140,1,"28,KB2",112,8,0,0
4,1077,1,1142,1,13.0,1142,1,"26,KB2",104,8,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
3768,4676,8,4172,29,45.0,4172,7,"-20,-8",-20,-8,0,0
3769,4677,8,4174,29,45.0,4174,7,"-28,-8",-28,-8,0,0
3770,4678,8,4176,29,45.0,4176,7,"-36,-8",-36,-8,0,0
3771,4679,8,4178,29,45.0,4178,7,"-44,-8",-44,-8,0,0


In [33]:
df = pd.merge(climbs.drop("angle", axis=1), stats, left_on="uuid", right_on="climb_uuid")
df['display_grade'] = df['display_difficulty'].apply(lambda x: grades.loc[int(x) + 1, "boulder_name"])
df['average_grade'] = df['difficulty_average'].apply(lambda x: grades.loc[int(x) + 1, "boulder_name"])
df = df[df['layout_id'] == 1]
df = df[df['frames_count'] == 1]
df = df[df['ascensionist_count'] > 3].reset_index()
# holds = holds[holds['product_id'] == 1]

In [35]:
holds

,layout_id,hole_id,set_id,default_placement_role_id,id_y,product_id,name,x,y,mirrored_hole_id,mirror_group
id_x,,,,,,,,,,,
1073,1,1134,1,13.0,1134,1,"34,KB2",136,8,0,0
1074,1,1136,1,13.0,1136,1,"32,KB2",128,8,0,0
1075,1,1138,1,13.0,1138,1,"30,KB2",120,8,0,0
1076,1,1140,1,13.0,1140,1,"28,KB2",112,8,0,0
1077,1,1142,1,13.0,1142,1,"26,KB2",104,8,0,0
...,...,...,...,...,...,...,...,...,...,...,...
4676,8,4172,29,45.0,4172,7,"-20,-8",-20,-8,0,0
4677,8,4174,29,45.0,4174,7,"-28,-8",-28,-8,0,0
4678,8,4176,29,45.0,4176,7,"-36,-8",-36,-8,0,0


In [40]:
swooped = df[df["name"] == "swooped"].iloc[0]["frames"]


def encode_frame(frames: str, angle: int, difficulty: float) -> torch.Tensor:
    matrix = torch.zeros((36, 44), dtype=torch.long)
    for frame in frames.split("p")[1:]:
        hold_id, color = frame.split("r")
        hold_id, color = int(hold_id), int(color) - 12
        try:
            hold = holds.loc[hold_id]
        except:
            print("Cant' find hold", hold_id, color)
            continue
        x, y = (hold.x - 4) // 4, (hold.y - 4) // 4
        matrix[x, y] = color +1
    return matrix

index = 300
print(df.iloc[index]['frames'], df.iloc[index]['name'])
px.imshow(encode_frame(df.loc[index, "frames"], 40, 10).t(), origin="lower", title=df.iloc[index]['name'])

p1165r12p1203r13p1254r13p1273r13p1280r15p1320r13p1339r13p1387r13p1403r13p1415r14p1434r14p1505r15p1508r15p1517r15p1527r15p1536r15p1559r15p1567r15p1576r15p1578r15p1584r15 Twist & Shout


In [95]:
holds

,id,product_id,name,x,y,mirrored_hole_id,mirror_group,hole_id
hold_id,,,,,,,,
1560,1133,1,"35,KB1",140,4,0,0,1133
1033,1134,1,"34,KB2",136,8,0,0,1134
1561,1135,1,"33,KB1",132,4,0,0,1135
1034,1136,1,"32,KB2",128,8,0,0,1136
1562,1137,1,"31,KB1",124,4,0,0,1137
...,...,...,...,...,...,...,...,...
4475,4422,1,"148,116",148,116,0,0,4422
4476,4423,1,"164,116",164,116,0,0,4423
4477,4424,1,"156,124",156,124,0,0,4424


In [9]:
subset = holds[holds['product_id'] == 1]
subset = subset[subset.index.to_series() < 3000]
px.scatter(subset[["x", "y"]]/4, x="x", y="y", hover_name=subset.index, color=subset.index)

In [13]:
used_holds = set()
for _, row in df.iterrows():
    for hold in row['frames'].split("p")[1:]:
        used_holds.add(int(hold.split("r")[0]))

In [14]:
used_holds

{1073,
 1074,
 1075,
 1076,
 1077,
 1078,
 1079,
 1080,
 1081,
 1082,
 1083,
 1084,
 1085,
 1086,
 1087,
 1088,
 1089,
 1090,
 1091,
 1092,
 1093,
 1094,
 1095,
 1096,
 1097,
 1098,
 1099,
 1100,
 1101,
 1102,
 1103,
 1104,
 1105,
 1106,
 1107,
 1108,
 1109,
 1110,
 1111,
 1112,
 1113,
 1114,
 1115,
 1116,
 1117,
 1118,
 1119,
 1120,
 1121,
 1122,
 1123,
 1124,
 1125,
 1126,
 1127,
 1128,
 1129,
 1130,
 1131,
 1132,
 1133,
 1134,
 1135,
 1136,
 1137,
 1138,
 1139,
 1140,
 1141,
 1142,
 1143,
 1144,
 1145,
 1146,
 1147,
 1148,
 1149,
 1150,
 1151,
 1152,
 1153,
 1154,
 1155,
 1156,
 1157,
 1158,
 1159,
 1160,
 1161,
 1162,
 1163,
 1164,
 1165,
 1166,
 1167,
 1168,
 1169,
 1170,
 1171,
 1172,
 1173,
 1174,
 1175,
 1176,
 1177,
 1178,
 1179,
 1180,
 1181,
 1182,
 1183,
 1184,
 1185,
 1186,
 1187,
 1188,
 1189,
 1190,
 1191,
 1192,
 1193,
 1194,
 1195,
 1196,
 1197,
 1198,
 1199,
 1200,
 1201,
 1202,
 1203,
 1204,
 1205,
 1206,
 1207,
 1208,
 1209,
 1210,
 1211,
 1212,
 1213,
 1214,
 1215,